In [1]:
import os, sys 
import torch 
import datetime
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import IPython.display as ipd
from argparse import Namespace
import torch.nn as nn 
import torch.nn.functional as F
import librosa
import argparse
import scipy 
from IPython.display import Audio
import numpy as np
from matplotlib.pyplot import cm
from tqdm import tqdm
import time

sys.path.append("../../")

In [2]:
from __config__ import *
import attentional_control.dnn.models.simplified_tasnet as simplified_tasnet
import attentional_control.dnn.losses.sisdr as sisdr_lib

os.environ['CUDA_VISIBLE_DEVICES'] = '3'
torch.cuda.empty_cache()

In [3]:
def get_model_requirements(model, opt,
                           number_of_batches=10,
                           batch_size=1,
                           n_sources=2,
                           time_length=32000,
                           fs=8000,
                           to_GPU=True):
    
    per_sec_c = number_of_batches * batch_size * n_sources * (time_length / (1. * fs))
    
    def random_input_gen(number_of_batches, batch_size, n_sources, time_length):
        for i in range(number_of_batches):
            sources = torch.rand(batch_size, n_sources, time_length)
            mixture = torch.sum(sources, dim=1)
            yield mixture, sources
            
    requirements_dict = {
        'Fetching': 0.,
        'Moving to GPU': 0.,
        'Train Forward': 0.,
        'Eval Forward': 0.,
        'Loss Computation': 0.,
        'Backward': 0.,
        'Parameters': 0.,
    }

    numparams = 0
    for f in model.parameters():
        if f.requires_grad:
            numparams += f.numel()
    requirements_dict['Parameters'] = numparams
            
    if to_GPU:
        model = model.cuda()

    sisdr_loss = sisdr_lib.PermInvariantSISDR(batch_size=batch_size,
                                              n_sources=n_sources,
                                              zero_mean=True,
                                              backward_loss=False,
                                              improvement=True,
                                              return_individual_results=False)

    train_gen = random_input_gen(number_of_batches, batch_size, n_sources, time_length)
    test_gen = random_input_gen(number_of_batches, batch_size, n_sources, time_length)

    
    model.train()

    for data in tqdm(train_gen, desc='Training'):
        st = time.time()
        opt.zero_grad()
        m1wavs = data[0].unsqueeze(1)
        clean_wavs = data[-1]
        requirements_dict['Fetching'] += (time.time() - st) / per_sec_c
        
        st = time.time()
        if to_GPU:
            m1wavs = m1wavs.cuda()
            clean_wavs = clean_wavs.cuda()
        requirements_dict['Moving to GPU'] += (time.time() - st) / per_sec_c
        
        st = time.time()
        rec_sources_wavs = model(m1wavs)
        requirements_dict['Train Forward'] += (time.time() - st) / per_sec_c

        st = time.time()
        l = sisdr_loss(rec_sources_wavs, clean_wavs, initial_mixtures=m1wavs)
        requirements_dict['Loss Computation'] += (time.time() - st) / per_sec_c
        
        st = time.time()
        l.backward()
        opt.step()
        requirements_dict['Backward'] += (time.time() - st) / per_sec_c
        
        
    del model
    return requirements_dict
    


In [ ]:
computational_results = {}

for P, R in [(3, 4), (3, 1), (3, 2), (61, 4), (61, 2), (61, 1)]:
    for to_GPU in [False, True]:
        hparams = {
            'B': 256,
            'H': 512,
            'P': P,
            'R': R,
            'X': 8,
            'L': 21,
            'N': 64,
            'S': 2,
            'lr': 0.001,
        }

        model = simplified_tasnet.TDCN(
            B=hparams['B'],
            H=hparams['H'],
            P=hparams['P'],
            R=hparams['R'],
            X=hparams['X'],
            L=hparams['L'],
            N=hparams['N'],
            S=hparams['S'])

        opt = torch.optim.Adam(model.parameters(), lr=hparams['lr'])

        reqs = get_model_requirements(model, opt, 
                                      number_of_batches=10,
                                      batch_size=1,
                                      n_sources=2,
                                      time_length=32000,
                                      to_GPU=to_GPU)
        
        computational_results['P_{}_R_{}_GPU_{}'.format(P, R, to_GPU)] = reqs

Training: 3it [02:00, 39.41s/it]

In [ ]:
print(computational_results)